<a href="https://colab.research.google.com/github/Thekowo/InfoLing-SchenkKlitschkov/blob/main/Crypto_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the latest master of Haystack
## You have to restart the runtime after this is complete! The Notebook should alert you of this.
!pip install grpcio-tools
!pip install git+https://github.com/deepset-ai/haystack.git


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import requests

In [ ]:
## This cell grabs all questions from an online article that answers the most googled questions about crypto
url = 'https://time.com/nextadvisor/investing/cryptocurrency/most-googled-crypto-questions-answered'
page=requests.get(url)
soup = BeautifulSoup(page.text)
content = soup.body
questions=[]
for h in content.find_all('h3'):
     question=h.text
     if('?' in question):
        questions.append(h.text)

pars=[]
for x in content.find_all('p'):
     answer=x.text
     if((len(answer))>20):
      pars.append(answer)
all_answers = pars[0:53]

f = open("doc.txt", "w")
for i in range(len(all_answers)):
    f.write(all_answers[i] + '\n')
f.close()

In [ ]:
url = 'https://time.com/nextadvisor/investing/cryptocurrency/most-googled-crypto-questions-answered'
page=requests.get(url)
soup = BeautifulSoup(page.text)
content = soup.body

In [ ]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
document_store.delete_all_documents

<bound method ElasticsearchDocumentStore.delete_all_documents of <haystack.document_store.elasticsearch.ElasticsearchDocumentStore object at 0x7f2ec899f790>>

In [ ]:
# txt. file ordner in dem angegeben path ablegen!
file_path= 'txt_files.zip'
path = 'data/crypto'


from zipfile import ZipFile
with ZipFile(file_path, 'r') as zipObj:
  zipObj.extractall(path)
import os

wrong_path= 'data/crypto/txt_files/~BROMIUM'
import sys
import shutil

# unzipping the file in collab creates some additional directory called bromium, this should be deleted:
# Get directory name
mydir= wrong_path
#delete the entire bromium directory
try:
    shutil.rmtree(mydir)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))

docs = path

In [ ]:
## this cell converts the documents and writes them, with light preprocessing 

## THIS IS THE DEFAULT IMPLEMENTATION

dicts = convert_files_to_dicts(dir_path=docs, clean_func=clean_wiki_text, split_paragraphs=True)

# writing the dicts to our document store
document_store.write_documents(dicts)

In [ ]:
## if splitting is required, this is the cell that splits the documents, and then writes to document store

## THIS IS NOT THE DEFAULT IMPLEMENTATION

from haystack.preprocessor import PreProcessor
processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=300,
    split_respect_sentence_boundary=True,
    split_overlap=0
)


# Convert files to dicts, with the already pre defined clean_wiki_text cleaning function
dicts = convert_files_to_dicts(dir_path=docs, clean_func=clean_wiki_text, split_paragraphs=True)

new_array=[]
for i in dicts:
  x = processor.process(i)
  for j in x:
    new_array.append(j)
document_store.write_documents(new_array)

One or more sentence found with word count higher than the split length.


In [ ]:
# Load a model that is suitable for the task - we chose roberta-base-squad2)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

from haystack.retriever.sparse import ElasticsearchRetriever
# ExtractiveQAPipeline lets you specify a retriever and a reader, and extracts answers for queries
from haystack.pipeline import ExtractiveQAPipeline, GenerativeQAPipeline

Some weights of the model checkpoint at deepset/roberta-base-squad2 were not used when initializing RobertaModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
ML Logging is turned off. No parameters, metrics or artifacts will be logged to 

In [ ]:
#die trainingsdatei answers.json muss wurde hier in den data ordner eingefügt
data_dir = "data"
reader.train(data_dir=data_dir, train_filename="answers.json", use_gpu=True, n_epochs=1, save_dir="my_model")

Train epoch 0/0 (Cur. train loss: 0.0026): 100%|██████████| 24/24 [00:21<00:00,  1.12it/s]


In [ ]:
## THIS IS THE DEFAULT IMPLEMENTATION

## specifies which retriever to use, and establishes the pipe
retriever = ElasticsearchRetriever(document_store=document_store)
base_pipe = ExtractiveQAPipeline(reader,retriever)

In [ ]:
## THIS IS THE DEFAULT IMPLEMENTATION

# runs all questions through the model, saves them into a txt file
# You can configure how many candidates the reader and retriever shall return with the k params
# The higher top_k_retriever, the better (but also the slower) your answers. 
answers =[]
for i in range(len(questions)):
  answers.append(base_pipe.run(query=questions[i], params= {"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}) ) 
  
f = open("answers.txt", "w")
for i in range(len(answers)):
  f.write('\n' +questions[i]+ '\n\n')
  for y in range (len(answers[i]['answers'])):
      f.write((answers[i]['answers'][y]['answer'])+'\n')
f.close()

In [ ]:
## THIS IS NOT THE DEFAULT IMPLEMENTATION

## specifies which retriever to use, and establishes the pipe 
test_pipe = ExtractiveQAPipeline(reader, test_retriever)
test_retriever=ElasticsearchRetriever(document_store=test_store)

In [ ]:
## THIS IS NOT THE DEFAULT IMPLEMENTATION

# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
test_answers =[]
for i in range(len(questions)):
  test_answers.append(test_pipe.run(query=questions[i], params= {"Retriever": {"top_k": 15}, "Reader": {"top_k": 5}}) ) 
  
f = open("test.txt", "w")
for i in range(len(test_answers)):
  f.write('\n' +questions[i]+ '\n\n')
  for y in range (len(test_answers[i]['answers'])):
      f.write((test_answers[i]['answers'][y]['answer'])+'\n')
f.close()

In [ ]:
## the following blocks use whatever reader is declared above, but establish a document store with embeddings
## and an embedding retriever

from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.document_store.faiss import FAISSDocumentStore


## this is the document store for dense passage retrieval
embed_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
embed_store.write_documents(dicts)


In [ ]:
##  initialises the dense passage retriever 
from haystack.retriever.dense import DensePassageRetriever
embed_retriever = DensePassageRetriever(document_store=embed_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
# Updates the embeddings in the document store 
embed_store.update_embeddings(embed_retriever)

In [ ]:
## establishes the pipeline including the DPR retriever/embed retriever
#reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, embed_retriever)

In [ ]:
embed_answers=[]
for i in range(len(questions)):
  embed_answers.append(pipe.run(query=questions[i],  params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}))
                                          #params= {"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}) ) 
f = open("embed.txt", "w")
for i in range(len(embed_answers)):
  f.write('\n' +questions[i]+ '\n\n')
  for y in range (len(embed_answers[i]['answers'])):
      f.write((embed_answers[i]['answers'][y]['answer'])+'\n')
f.close()